<a href="https://colab.research.google.com/github/Qu1nnD/CS290/blob/main/GaussianDistribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Follow the examples in Chapter 9 of Hands-on Machine Learning to fit Gaussian Mixture Models to both the iris and penguins datasets. After fitting GMMs to these datasets,
Plot the centers of the Gaussian distributions in your GMM along with the centroids from your 𝑘-means modeling results. How different are they?
Calculate the overall accuracy of each GMM, and compare with the overall accuracy of your 𝑘-means models.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.datasets import load_iris

In [6]:
penguins = pd.read_csv("https://github.com/mbrudd/csci290/raw/refs/heads/main/data/penguins.csv")
iris = load_iris()
iris_data = pd.DataFrame( iris.data, columns = iris.feature_names)

In [ ]:
def ini_Centroids(k, dataset, centroid_type, features):
  if(centroid_type == "random"): # Makes the centroid random
    indices = np.random.choice(len(features), size=k, replace=False)
    return features.to_numpy()[indices]
  elif(centroid_type == 'arthur'): #arthur vestile method
    centroids = []
    first_centroid = features.sample(n=1)
    centroids.append(first_centroid.values[0])
    for x in range(1, k):
      distances = np.array([min(np.linalg.norm(x - np.array(centroid), axis=0) ** 2 for centroid in centroids) for x in features.to_numpy()])
      total_distance = distances.sum()
      probabilities = distances / total_distance  # Probability: D(x_i)^2 / sum(D(x_j)^2)
      chosen_idx = np.random.choice(len(features), p=probabilities)
      centroids.append(features.iloc[chosen_idx].values)
    return np.array(centroids)
  else: # Makes the centroid farthest away from everything
    centroids = [features.sample(n=1).values[0]]  # Choose the first centroid randomly
    for x in range(1, k):
        distances = np.array([min(np.linalg.norm(point - np.array(c)) for c in centroids)for point in features.to_numpy()])
        farthest_idx = np.argmax(distances)
        centroids.append(features.iloc[farthest_idx].values)
    return np.array(centroids)

In [ ]:
def accuracy(true_labels, predicted_labels):
    correct_predictions = np.sum(true_labels == predicted_labels)
    return correct_predictions / len(true_labels)

In [ ]:
def K_Means(k, dataset, centroid_type, feature_x, feature_y, target):
    converged = False
    features = dataset[[feature_x, feature_y]].dropna()
    centroids=ini_Centroids(k, dataset, centroid_type, features) # initializes the first centroids
    while not converged: # Loops until there isn't massive changes between centroids and new_centroids
        #Calculations
        distances = np.linalg.norm(features.to_numpy()[:, np.newaxis] - centroids, axis=2) # calcuates the eudclidean distance between data points and centroids
        labels = np.argmin(distances, axis=1) # assigns data points to the closest centroid
        new_centroids = np.array([features[labels == i].mean(axis=0) if np.any(labels == i) else centroids[i] for i in range(k)]) # calculates updated centroids given the mean of all data points from each cluster
        #Convergence test and updating centroids
        converged = np.all(np.abs(centroids - new_centroids) <= 0.001) # Tests to see if there is major differences between the current and new centroids to see if it needs to continue the loop again
        centroids = new_centroids
    label_mapping = {}
    for i in range(k):
        cluster_indices = np.where(labels == i)[0]
        if len(cluster_indices) > 0:
            majority_label = dataset.iloc[cluster_indices][target].mode().iloc[0]
            label_mapping[i] = majority_label
    # Replace numerical labels with string labels
    string_labels = [label_mapping[label] for label in labels]
    #Add making the predicted labels have a string value
    return string_labels, labels, centroids

In [56]:
def plot_GMM(k, dataset, features, target):
    features_data = dataset[features].dropna()
    target_data = dataset[target].iloc[features_data.index]

    # Fit GMM
    gmm = GaussianMixture(n_components=k, covariance_type='full', random_state=42)
    gmm.fit(features_data)
    labels = gmm.predict(features_data)

    # Generate grid for contour plot
    x_min, x_max = features_data[features[0]].min() - 1, features_data[features[0]].max() + 1
    y_min, y_max = features_data[features[1]].min() - 1, features_data[features[1]].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
    grid = np.c_[xx.ravel(), yy.ravel()]

    # Evaluate GMM probabilities on the grid
    probs = -gmm.score_samples(grid).reshape(xx.shape)  # Negative log-probabilities

    # Plot scatter and contour
    plt.figure(figsize=(8, 6))
    plt.scatter(features_data[features[0]], features_data[features[1]], c=labels, cmap='viridis', alpha=0.6, s=50)
    plt.contour(xx, yy, probs, levels=15, cmap='coolwarm', alpha=0.7)
    plt.title("Gaussian Mixture Model with Contour")
    plt.xlabel(features[0])
    plt.ylabel(features[1])
    plt.colorbar(label="Negative Log-Likelihood")
    plt.show()

    # Adjusted Rand Index (ARI) for evaluation
    ari = adjusted_rand_score(target_data, labels)

    return {
        "model": gmm,
        "labels": labels,
        "ari": ari
    }

Iris Dataset

In [8]:
gm1 = GaussianMixture(n_components=3, n_init=10)
gm1.fit(iris_data)

In [12]:
gm1.weights_

array([0.33333333, 0.30118609, 0.36548058])

In [13]:
gm1.means_

array([[5.006     , 3.428     , 1.462     , 0.246     ],
       [5.91697517, 2.77803998, 4.20523542, 1.29841561],
       [6.54632887, 2.94943079, 5.4834877 , 1.98716063]])

In [14]:
gm1.covariances_

array([[[0.121765  , 0.097232  , 0.016028  , 0.010124  ],
        [0.097232  , 0.140817  , 0.011464  , 0.009112  ],
        [0.016028  , 0.011464  , 0.029557  , 0.005948  ],
        [0.010124  , 0.009112  , 0.005948  , 0.010885  ]],

       [[0.27550587, 0.09663458, 0.18542939, 0.05476915],
        [0.09663458, 0.09255531, 0.09103836, 0.04299877],
        [0.18542939, 0.09103836, 0.20227635, 0.0616792 ],
        [0.05476915, 0.04299877, 0.0616792 , 0.03232217]],

       [[0.38741443, 0.09223101, 0.30244612, 0.06089936],
        [0.09223101, 0.11040631, 0.08386768, 0.0557538 ],
        [0.30244612, 0.08386768, 0.32595958, 0.07283247],
        [0.06089936, 0.0557538 , 0.07283247, 0.08488025]]])

In [15]:
gm1.converged_

True

In [16]:
gm1.n_iter_

17

In [18]:
gm1.predict(iris_data)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [19]:
gm1.predict_proba(iris_data).round(3)

array([[1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [1.

In [20]:
x_new,y_new=gm1.sample(6)

In [21]:
x_new

array([[4.58922524, 2.79910243, 1.19345172, 0.17584526],
       [5.51135662, 3.51349414, 1.5715402 , 0.32194197],
       [5.23104376, 2.95137421, 1.44717972, 0.21389719],
       [6.16166881, 2.46266367, 4.53473845, 1.27431668],
       [6.28584719, 3.34798244, 4.36050842, 1.37081118],
       [6.56318845, 3.21941987, 4.8535455 , 1.39813646]])

In [22]:
y_new

array([0, 0, 0, 1, 1, 1])

In [23]:
gm1.score_samples(iris_data).round(2)

array([ 1.57,  0.74,  1.14,  0.93,  1.41, -0.09,  0.05,  1.62,  0.27,
        0.17,  0.83,  0.77,  0.3 , -1.79, -3.42, -2.11, -1.13,  1.48,
       -0.85,  0.98, -0.93,  0.41, -3.84, -1.89, -3.17, -0.12,  0.51,
        1.38,  1.12,  0.69,  0.78, -0.69, -2.13, -0.88,  1.15,  0.12,
       -1.12,  0.23,  0.13,  1.5 ,  0.94, -4.49, -0.34, -4.48, -2.59,
        0.68,  0.39,  1.04,  1.16,  1.55, -2.04, -0.27, -0.85, -2.33,
       -1.16, -0.79, -0.82, -1.4 , -0.45, -1.64, -2.59, -0.6 , -2.52,
       -0.11, -1.93, -1.16, -1.27, -2.94, -5.17,  0.26, -2.49, -0.02,
       -2.19, -2.58,  0.09, -0.48, -1.33, -2.27, -0.06, -1.86,  0.13,
       -0.49,  0.54, -1.43, -2.52, -2.22, -0.32, -3.75, -0.44, -0.35,
       -1.84,  0.24,  0.7 , -1.18,  0.44, -1.74,  0.35,  0.59, -4.2 ,
        0.75, -4.19, -1.3 , -0.77, -1.39, -1.12, -2.55, -4.41, -2.35,
       -2.07, -2.53, -1.52, -0.62, -0.7 , -2.27, -3.41, -1.35, -0.8 ,
       -5.03, -7.06, -2.99, -1.04, -1.69, -3.85, -1.35, -0.95, -1.97,
       -1.41, -1.23,

In [ ]:
plot_GMM(3, iris_data, ["petal length (cm)", "petal width (cm)"], "target")

Penguins dataset

In [26]:
gm2 = GaussianMixture(n_components=3, n_init=10)
penguins = penguins[["flipper_length_mm","bill_length_mm"]]
penguins = penguins.dropna()
gm2.fit(penguins)

GaussianMixture(n_components=3, n_init=10)

In [27]:
gm2.weights_

array([0.20891229, 0.41972089, 0.37136681])

In [28]:
gm2.means_

array([[196.50939504,  48.67856476],
       [189.08913749,  38.60158257],
       [216.7595804 ,  47.25917307]])

In [29]:
gm2.covariances_

array([[[43.80771074,  6.84627505],
        [ 6.84627505, 11.06620647]],

       [[34.99100075,  2.73226839],
        [ 2.73226839,  6.38449399]],

       [[46.66558079, 16.11513628],
        [16.11513628, 10.73042564]]])

In [40]:
gm2.converged_

True

In [41]:
gm2.n_iter_

11

In [42]:
gm2.predict(penguins)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,

In [43]:
gm2.predict_proba(penguins).round(3)

array([[0.005, 0.995, 0.   ],
       [0.007, 0.993, 0.   ],
       [0.022, 0.968, 0.009],
       ...,
       [1.   , 0.   , 0.   ],
       [0.707, 0.   , 0.293],
       [1.   , 0.   , 0.   ]])

In [44]:
x_new2,y_new2=gm1.sample(6)

In [45]:
x_new2

array([[5.06253504, 3.50061866, 1.35746016, 0.26814042],
       [6.13887242, 2.68950029, 4.70341374, 1.41527226],
       [5.96455445, 2.50943852, 3.57598231, 1.19513608],
       [5.92252523, 2.73245277, 3.88918445, 1.19423581],
       [5.33377153, 3.02781987, 4.30363907, 1.76700911],
       [6.9621314 , 3.06064244, 6.17445166, 2.1211694 ]])

In [46]:
y_new2

array([0, 1, 1, 1, 2, 2])

In [48]:
gm2.score_samples(penguins).round(2)

array([ -6.43,  -5.63,  -5.98,  -6.  ,  -5.43,  -6.39,  -5.88,  -7.48,
        -6.2 ,  -5.55,  -6.57,  -6.83,  -5.44,  -8.31,  -5.86,  -5.89,
        -6.87,  -6.94,  -6.73,  -8.66,  -6.58,  -5.98,  -5.63,  -6.64,
        -6.25,  -6.4 ,  -5.78,  -9.6 ,  -7.11,  -7.4 ,  -7.17,  -5.48,
        -6.32,  -6.46,  -6.05,  -5.4 ,  -7.98,  -6.34,  -5.96,  -6.31,
        -6.09,  -5.98,  -7.02,  -5.76,  -5.47,  -6.83,  -6.85,  -5.98,
        -6.32,  -5.65,  -5.6 ,  -6.5 ,  -7.16,  -6.71,  -5.96,  -5.56,
        -5.81,  -6.5 ,  -5.89,  -6.17,  -6.23,  -5.67,  -5.89,  -6.03,
        -6.06,  -6.92,  -5.91,  -6.03,  -6.83,  -7.57,  -5.48,  -6.06,
        -6.83,  -6.22,  -6.71,  -5.79,  -5.84,  -5.86,  -6.48,  -6.69,
        -6.85,  -5.7 ,  -6.78,  -5.61,  -6.11,  -5.86,  -5.63,  -5.4 ,
        -5.4 ,  -8.96,  -7.29,  -7.12,  -5.65,  -5.86,  -7.48,  -5.43,
        -6.14,  -8.89,  -6.68,  -6.7 ,  -7.3 ,  -5.94,  -5.46,  -5.69,
        -5.94,  -6.78,  -5.42,  -6.33,  -7.03,  -6.61,  -6.93,  -5.65,
      

In [57]:
plot_GMM(3, penguins, ["flipper_length_mm","bill_length_mm"], "species")

KeyError: 'species'